In [1]:
import numpy as np
import os
import re
import cv2
from pathlib import Path
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate
from tensorflow.keras.layers import LeakyReLU

In [2]:
image_dir = '../dataset/augmented_imgs/'


In [3]:
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)

In [4]:
SIZE = 256
high_img = []
path = '../dataset/augmented_imgs/high_res'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):    
    # if i == '855.png':
    #     break
    # else:    
    img = cv2.imread(path + '/'+i,1)        
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (SIZE, SIZE))
    img = img.astype('float32') / 255.0
    high_img.append(img_to_array(img))

100%|██████████| 8550/8550 [05:03<00:00, 28.20it/s]  


In [5]:
low_img = []
path = '../dataset/augmented_imgs/denoised_low_res'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
    # if i == '855.png':
    #     break
    # else: 
    img = cv2.imread(path + '/'+i,1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #resizing image
    img = cv2.resize(img, (SIZE, SIZE))
    img = img.astype('float32') / 255.0
    low_img.append(img_to_array(img))

100%|██████████| 4275/4275 [03:41<00:00, 19.28it/s]


In [6]:
# high_img = np.array(high_img)
# low_img = np.array(low_img)

In [7]:

# Splitting the datasets
train_high_image = high_img[:3400]
train_low_image = low_img[:3400]

validation_high_image = high_img[3400:3800]
validation_low_image = low_img[3400:3800]

test_high_image = high_img[3800:]
test_low_image = low_img[3800:]

In [8]:

train_high_image = np.reshape(train_high_image, (len(train_high_image), SIZE, SIZE, 3))
train_low_image = np.reshape(train_low_image, (len(train_low_image), SIZE, SIZE, 3))

validation_high_image = np.reshape(validation_high_image, (len(validation_high_image), SIZE, SIZE, 3))
validation_low_image = np.reshape(validation_low_image, (len(validation_low_image), SIZE, SIZE, 3))

test_high_image = np.reshape(test_high_image, (len(test_high_image), SIZE, SIZE, 3))
test_low_image = np.reshape(test_low_image, (len(test_low_image), SIZE, SIZE, 3))

MemoryError: Unable to allocate 2.49 GiB for an array with shape (3400, 256, 256, 3) and data type float32

In [11]:
# Print shapes for verification
print("Shape of training high-resolution images:", train_high_image.shape)
print("Shape of training low-resolution images:", train_low_image.shape)
print("Shape of validation high-resolution images:", validation_high_image.shape)
print("Shape of validation low-resolution images:", validation_low_image.shape)
print("Shape of test high-resolution images:", test_high_image.shape)
print("Shape of test low-resolution images:", test_low_image.shape)

Shape of training high-resolution images: (3400, 256, 256, 3)
Shape of training low-resolution images: (3400, 256, 256, 3)
Shape of validation high-resolution images: (400, 256, 256, 3)
Shape of validation low-resolution images: (400, 256, 256, 3)
Shape of test high-resolution images: (474, 256, 256, 3)
Shape of test low-resolution images: (473, 256, 256, 3)


In [17]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)    
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  
    x = Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape):
    inputs = Input(shape=(256,256,3))
    
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)             #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    # outputs = Conv2D(3, 1, padding="same", activation="LeakyReLU")(d4)
    outputs = Conv2D(3, 1, padding="same")(d4)
    outputs = LeakyReLU()(outputs)

    model = Model(inputs, outputs, name="U-Net")
    
    # model.summary()
    return model

In [18]:
inputs = Input(shape=(256,256,3))
model = build_unet(inputs)

In [19]:
model.save('srmodel.h5')